In [1]:
from os import environ
from iexfinance.stocks import Stock
from dotenv import load_dotenv
from datetime import datetime
from iexfinance.stocks import Stock
import pyodbc
import sqlalchemy
import pandas
import urllib
from sqlalchemy import create_engine

# splits a list into chunks of the given size
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# load the environment variables and verity we have a token
load_dotenv()
iex_token = [environ[key] for key in environ if key == 'IEX_TOKEN']
print(iex_token[0][0:10])

Tsk_5c3a24


In [2]:
# connect to the database
drivers = [x for x in pyodbc.drivers() if x.endswith(' for SQL Server')]
print(drivers)
(svr, db, uid, pwd) = environ['MSSQL_SERVER'], environ['MSSQL_DB'], environ['MSSQL_UID'], environ['MSSQL_PWD']
conn = pyodbc.connect(f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={svr};DATABASE={db};UID={uid};PWD={pwd}')

['ODBC Driver 17 for SQL Server']


In [3]:
# get the symbols
sql = "Exec usp_Equity_Read"
symbols = pandas.read_sql(sql,conn)
symbols['Symbol'] = symbols['Symbol'].str.upper()
symbol_list = symbols['Symbol'].tolist()
symbols.head()

,EquityID,Symbol,LastProcessDate
0,3,AAPL,2020-12-21 17:20:10.080
1,13,ADBE,2020-12-19 18:40:09.963
2,32,AMD,2020-12-19 19:50:08.020
3,34,AMGN,2020-12-22 19:00:08.537
4,125,CGNX,2020-12-20 04:50:14.110


In [4]:
# get the trade date id
# today = datetime.today().strftime('%Y-%m-%d')
today = '2020-12-16'
sql = f"SELECT TradedateID FROM dbo.TradeDate WHERE TradeDate = '{today}'"
cursor = conn.cursor()
cursor.execute(sql)
tradeDateID = cursor.fetchone()[0]

In [5]:
# pull the data from iex and add it to the data frame
# iex can only get 100 records at a time
chunks = chunker(symbol_list, 100)
data = symbols[['EquityID', 'Symbol']]
quotes = []
for chunk in chunks:
    batch = Stock(chunk, output_format="pandas")
    quote = batch.get_quote()
    quote = quote[['symbol', 'open', 'close', 'high', 'low', 'volume']]
    quotes.append(quote)
quotes = pandas.concat(quotes)
data = data.merge(quotes, left_on='Symbol', right_on='symbol', how='outer')
data.head()

,EquityID,Symbol,symbol,open,close,high,low,volume
0,3,AAPL,AAPL,136.17,137.93,136.99,130.58,172394164
1,13,ADBE,ADBE,507.64,528.17,513.58,500.08,1712307
2,32,AMD,AMD,94.521,96.01,97.99,92.66,37302121
3,34,AMGN,AMGN,228.37,223.08,230.175,224.27,3714214
4,125,CGNX,CGNX,84.4,81.78,84.905,80.22,1186499


In [6]:
# make the columns match the database table
data.rename(columns = {
    'open': 'Open', 
    'close': 'Close', 
    'high': 'High',
    'low': 'Low',
    'volume': 'Volume'
}, inplace = True) 
data.drop(columns=['symbol'], inplace = True)
data['TradeDate'] = today
data['TradeDateID'] = tradeDateID
data['LastModifiedDate'] = datetime.now()
data.head()

,EquityID,Symbol,Open,Close,High,Low,Volume,TradeDate,TradeDateID,LastModifiedDate
0,3,AAPL,136.17,137.93,136.99,130.58,172394164,2020-12-16,13515,2020-12-22 23:08:54.085683
1,13,ADBE,507.64,528.17,513.58,500.08,1712307,2020-12-16,13515,2020-12-22 23:08:54.085683
2,32,AMD,94.521,96.01,97.99,92.66,37302121,2020-12-16,13515,2020-12-22 23:08:54.085683
3,34,AMGN,228.37,223.08,230.175,224.27,3714214,2020-12-16,13515,2020-12-22 23:08:54.085683
4,125,CGNX,84.4,81.78,84.905,80.22,1186499,2020-12-16,13515,2020-12-22 23:08:54.085683


In [7]:
# write to the database
(svr, db, uid, pwd) = environ['MSSQL_SERVER'], environ['MSSQL_DB'], environ['MSSQL_UID'], environ['MSSQL_PWD']
conn = pyodbc.connect(f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={svr};DATABASE={db};UID={uid};PWD={pwd}')

quoted = urllib.parse.quote_plus(f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={svr};DATABASE={db};UID={uid};PWD={pwd}')
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={quoted}')

data.to_sql('Stage_IEXData', schema='dbo', con=engine, if_exists='append', index=False)